In [1]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir -p /content/drive/My\ Drive/nn_output

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
OUTPUTDIR='/content/drive/My Drive/nn_output'

In [66]:
!pip install keras-TCN

from keras.layers import (Bidirectional, Dense, Embedding, Input, Lambda, InputLayer, Reshape
                          , LSTM, RepeatVector, TimeDistributed, Flatten, Layer, GRU)
from keras.models import Model, Sequential, load_model
from tcn import TCN
from keras.utils import to_categorical
from keras import optimizers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import Constant
import numpy as np
from nltk.corpus import reuters
from itertools import chain
import nltk
nltk.download('reuters')
nltk.download('punkt')
from keras.callbacks import ModelCheckpoint
import os.path
import glob

USE_GLOVE = True
MAX_SEQUENCE_LEN = 100
MAX_NUM_WORDS = 10000

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
from itertools import zip_longest
def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    args = [iter(iterable)] * n
    return zip_longest(fillvalue=fillvalue, *args)

class ReutersGenerator():
    def __init__(self, max_seq_length=250, num_words=5000):
        self.tok = Tokenizer(num_words=num_words)
        self.max_seq_length = max_seq_length
        self.num_words = num_words
    
    def _gen_sents(self, fids):
        return (' '.join(sent) for fid in fids for sent in reuters.sents(fid))
    
    def fit(self, fid_startswith='train'):
        fids = (fid for fid in reuters.fileids() if fid.startswith(fid_startswith))
        self.tok.fit_on_texts(self._gen_sents(fids))
        return self

    def count(self, fid_startswith='train'):
        fids = (fid for fid in reuters.fileids() if fid.startswith(fid_startswith))
        return sum(1 for _ in self._gen_sents(fids))
    
    def inverse_transform(self, X):
        return self.tok.sequences_to_texts(X)
    
    def generate_pairs(self, fid_startswith='train', bs=32, 
                         max_seq_len=250, forever=True, shuffle=True):
        fids_in = np.array([fid for fid in reuters.fileids() if fid.startswith(fid_startswith)])
        index = np.arange(fids_in.shape[0])
        while True:
            np.random.shuffle(index)
            fids = fids_in[index]
            sents = self._gen_sents(fids)
            for batch in grouper(sents, bs):
                seqs = self.tok.texts_to_sequences_generator(text for text in batch if text)
                X = pad_sequences(list(seqs), self.max_seq_length)
                yield X, to_categorical(X, self.num_words)
            if not forever:
                break

In [0]:
reuters_gen = ReutersGenerator(
    num_words=MAX_NUM_WORDS, max_seq_length=MAX_SEQUENCE_LEN).fit()
n_train_full = reuters_gen.count('train')
#n_test = reuters_gen.count('test')

In [6]:
if not os.path.isfile("glove.6B.100d.txt"):
  !wget "http://nlp.stanford.edu/data/glove.6B.zip"
  !unzip "glove.6B.zip"

# get glove coeff matrix
embeddings_index = {}
with open("glove.6B.100d.txt", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Found %s word vectors.' % len(embeddings_index))

# prepare pre-learned embedding matrix
embdedding_dim = 100
word_index = reuters_gen.tok.word_index
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, embdedding_dim))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

--2019-01-17 07:42:30--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-01-17 07:42:30--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  94.6MB/s    in 9.3s    

2019-01-17 07:42:39 (88.5 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       
Found 400000 word vectors.


In [0]:
import keras.backend as K
import keras.layers
from keras import optimizers
from keras.engine.topology import Layer
from keras.layers import Activation, Lambda
from keras.layers import Conv1D, SpatialDropout1D
from keras.layers import Convolution1D, Dense
from keras.models import Input, Model
from typing import List, Tuple


def channel_normalization(x):
    # type: (Layer) -> Layer
    """ Normalize a layer to the maximum activation
    This keeps a layers values between zero and one.
    It helps with relu's unbounded activation
    Args:
        x: The layer to normalize
    Returns:
        A maximal normalized layer
    """
    max_values = K.max(K.abs(x), 2, keepdims=True) + 1e-5
    out = x / max_values
    return out


def wave_net_activation(x):
    # type: (Layer) -> Layer
    """This method defines the activation used for WaveNet
    described in https://deepmind.com/blog/wavenet-generative-model-raw-audio/
    Args:
        x: The layer we want to apply the activation to
    Returns:
        A new layer with the wavenet activation applied
    """
    tanh_out = Activation('tanh')(x)
    sigm_out = Activation('sigmoid')(x)
    return keras.layers.multiply([tanh_out, sigm_out])


def residual_block(x, s, i, activation, nb_filters, kernel_size, padding, dropout_rate=0, name=''):
    # type: (Layer, int, int, str, int, int, str, float, str) -> Tuple[Layer, Layer]
    """Defines the residual block for the WaveNet TCN
    Args:
        x: The previous layer in the model
        s: The stack index i.e. which stack in the overall TCN
        i: The dilation power of 2 we are using for this residual block
        activation: The name of the type of activation to use
        nb_filters: The number of convolutional filters to use in this block
        kernel_size: The size of the convolutional kernel
        padding: The padding used in the convolutional layers, 'same' or 'causal'.
        dropout_rate: Float between 0 and 1. Fraction of the input units to drop.
        name: Name of the model. Useful when having multiple TCN.
    Returns:
        A tuple where the first element is the residual model layer, and the second
        is the skip connection.
    """

    original_x = x
    conv = Conv1D(filters=nb_filters, kernel_size=kernel_size,
                  dilation_rate=i, padding=padding,
                  name=name + '_d_%s_conv_%d_tanh_s%d' % (padding, i, s))(x)
    if activation == 'norm_relu':
        x = Activation('relu')(conv)
        x = Lambda(channel_normalization)(x)
    elif activation == 'wavenet':
        x = wave_net_activation(conv)
    else:
        x = Activation(activation)(conv)

    x = SpatialDropout1D(dropout_rate, name=name + '_spatial_dropout1d_%d_s%d_%f' % (i, s, dropout_rate))(x)

    # 1x1 conv.
    x = Convolution1D(nb_filters, 1, padding='same')(x)
    res_x = keras.layers.add([original_x, x])
    return res_x, x


def process_dilations(dilations):
    def is_power_of_two(num):
        return num != 0 and ((num & (num - 1)) == 0)

    if all([is_power_of_two(i) for i in dilations]):
        return dilations

    else:
        new_dilations = [2 ** i for i in dilations]
        # print(f'Updated dilations from {dilations} to {new_dilations} because of backwards compatibility.')
        return new_dilations


class TCN:
    """Creates a TCN layer.
        Input shape:
            A tensor of shape (batch_size, timesteps, input_dim).
        Args:
            nb_filters: The number of filters to use in the convolutional layers.
            kernel_size: The size of the kernel to use in each convolutional layer.
            dilations: The list of the dilations. Example is: [1, 2, 4, 8, 16, 32, 64].
            nb_stacks : The number of stacks of residual blocks to use.
            activation: The activations to use (norm_relu, wavenet, relu...).
            padding: The padding to use in the convolutional layers, 'causal' or 'same'.
            use_skip_connections: Boolean. If we want to add skip connections from input to each residual block.
            return_sequences: Boolean. Whether to return the last output in the output sequence, or the full sequence.
            dropout_rate: Float between 0 and 1. Fraction of the input units to drop.
            name: Name of the model. Useful when having multiple TCN.
        Returns:
            A TCN layer.
        """

    def __init__(self,
                 nb_filters=64,
                 kernel_size=2,
                 nb_stacks=1,
                 dilations=[1, 2, 4, 8, 16, 32],
                 activation='norm_relu',
                 padding='causal',
                 use_skip_connections=True,
                 dropout_rate=0.0,
                 return_sequences=True,
                 name='tcn'):
        self.name = name
        self.return_sequences = return_sequences
        self.dropout_rate = dropout_rate
        self.use_skip_connections = use_skip_connections
        self.activation = activation
        self.dilations = dilations
        self.nb_stacks = nb_stacks
        self.kernel_size = kernel_size
        self.nb_filters = nb_filters
        self.padding = padding

        if padding != 'causal' and padding != 'same':
            raise ValueError("Only 'causal' or 'same' padding are compatible for this layer.")

        if not isinstance(nb_filters, int):
            print('An interface change occurred after the version 2.1.2.')
            print('Before: tcn.TCN(i, return_sequences=False, ...)')
            print('Now should be: tcn.TCN(return_sequences=False, ...)(i)')
            print('Second solution is to pip install keras-tcn==2.1.2 to downgrade.')
            raise Exception()

    def __call__(self, inputs):
        x = inputs
        x = Convolution1D(self.nb_filters, 1, padding=self.padding, name=self.name + '_initial_conv')(x)
        skip_connections = []
        for s in range(self.nb_stacks):
            for i in self.dilations:
                x, skip_out = residual_block(x, s, i, self.activation, self.nb_filters,
                                             self.kernel_size, self.padding, self.dropout_rate, name=self.name)
                skip_connections.append(skip_out)
        if self.use_skip_connections:
            x = keras.layers.add(skip_connections)
        x = Activation('relu', name=self.name)(x)

        if not self.return_sequences:
            output_slice_index = -1
            x = Lambda(lambda tt: tt[:, output_slice_index, :])(x)
        return x


In [77]:
kernel_size = 3
n_dilations = 8
n_hidden = 256
embedding_size = 100
dropout=0.4
encoder_type = 'GRU'

TRAIN_MODEL = True
BATCH_SIZE = 32
EPOCHS = 1

input_layer = Input(shape=(MAX_SEQUENCE_LEN,))
encoder = None
if not USE_GLOVE:
  encoder = Embedding(MAX_NUM_WORDS, embedding_size)(input_layer)
else:
  encoder = Embedding(num_words, embdedding_dim, 
                      input_length=MAX_SEQUENCE_LEN, 
                      embeddings_initializer=Constant(embedding_matrix),
                      trainable=True)(input_layer)
encoders = dict(
  TCN = TCN(name='tempconv1', return_sequences=True,
              kernel_size=kernel_size,
              dilations=[2**n for n in range(n_dilations)],
              nb_filters=n_hidden,
              nb_stacks=1,
              dropout_rate=dropout),
  LSTM = LSTM(n_hidden, name='tempconv1', return_sequences=True),
  GRU = GRU(n_hidden, name='tempconv1', return_sequences=True)
)
encoder = encoders[encoder_type](encoder)
#encoder = TCN(name='tempconv2', return_sequences=True,
#              kernel_size=kernel_size,
#              dilations=[2**n for n in range(n_dilations)],
#              nb_filters=n_hidden,
#              nb_stacks=1,
#              dropout_rate=dropout)(encoder)
output_layer = TimeDistributed(Dense(MAX_NUM_WORDS, activation='softmax'))(encoder)
model = Model(input_layer, output_layer)
optimizer = optimizers.Adam(lr=0.002, clipnorm=0.4)
model.compile(optimizer=optimizer, metrics=['accuracy'], loss='categorical_crossentropy')
#print(model.summary())

basename = 'seq2seq-%s-model-tl' % encoder_type

outfname = os.path.join(
    OUTPUTDIR,
    basename + '-ep{epoch:02d}.hdf5')
cp = ModelCheckpoint(
    outfname,
    save_best_only=False,
    save_weights_only=False)

if TRAIN_MODEL:
  history = model.fit_generator(reuters_gen.generate_pairs('train', bs=BATCH_SIZE),
      #validation_data=reuters_gen.generate_pairs('test', bs=BATCH_SIZE),
      steps_per_epoch=n_train_full//BATCH_SIZE,
      #validation_steps=n_test//BATCH_SIZE,
      epochs=EPOCHS, shuffle=True, callbacks=[cp])
else:
  list_of_files = glob.glob(os.path.join(OUTPUTDIR, basename + '*.hdf5'))
  list_of_files = sorted(list_of_files, key=os.path.getctime)
  assert(len(list_of_files) > 0)
  model = load_model(list_of_files[-1])
  print('Loaded model from \'%s\'' % list_of_files[-1])
  
X_test, X_test_hat = next(reuters_gen.generate_pairs('test'))
print(repr(reuters_gen.inverse_transform(np.argmax(model.predict(X_test[:10], verbose=1), axis=2))))
print(repr(reuters_gen.inverse_transform(X_test[:10])))

Epoch 1/1
10/10 [==============================] - 5s 467ms/step
['fundamental brokers buys part of brokers fundamental brokers institutional associates a leading inter dealer broker in u s government securities said it has agreed to acquire certain assets of government brokers inc for undisclosed terms', 'acquisition documents have been signed and are being held in escrow pending the receipt of certain it said', 'in conjunction with the sale of assets has treasury bills notes and bonds fundamental said', 'industry sources told reuters yesterday that fundamental was close to acquiring the government securities brokerage division of a major broker of corporate bonds', 'fundamental said it intends to use the facilities formerly used by to provide a new block service in the most active treasury issues', 'by the execution of wholesale trades from the heavy volume of smaller lots large scale transactions will be the company said', 'the new system is expected to substantially enhance the liq

Epoch 1/10
1258/1258 [==============================] - 282s 224ms/step - loss: 0.3309 - acc: 0.9592
Epoch 2/10
1258/1258 [==============================] - 277s 220ms/step - loss: 0.0610 - acc: 0.9933
Epoch 3/10
1258/1258 [==============================] - 275s 219ms/step - loss: 0.0412 - acc: 0.9962
Epoch 4/10
1258/1258 [==============================] - 275s 218ms/step - loss: 0.0375 - acc: 0.9969
Epoch 5/10
1258/1258 [==============================] - 275s 219ms/step - loss: 0.0357 - acc: 0.9972
Epoch 6/10
1258/1258 [==============================] - 275s 219ms/step - loss: 0.0348 - acc: 0.9974
Epoch 7/10
1258/1258 [==============================] - 276s 219ms/step - loss: 0.0340 - acc: 0.9976
Epoch 8/10
1258/1258 [==============================] - 275s 219ms/step - loss: 0.0342 - acc: 0.9976
Epoch 9/10
1258/1258 [==============================] - 276s 220ms/step - loss: 0.0332 - acc: 0.9977
Epoch 10/10
1258/1258 [==============================] - 275s 219ms/step - loss: 0.0327 - a

10/10 [==============================] - 1s 67ms/step


['allwaste lt o to acquire firm allwaste inc said it entered into an agreement in principle to buy co a privately held company that in exchange for about 1 3 mln shares or allwaste common',
 'allwaste said that earned about one mln dlrs before taxes last year and had about 1 5 mln dlrs in pre tax income for the nine months ended september 30',
 'fed data suggest no change in monetary policy new u s banking data suggest the federal reserve is monetary policy along a steady path and is not any imminent change of course economists said',
 "but they also said that if money supply growth remains weak as this week ' s anticipating eight billion dlr m 1 decline suggests it may this could influence the fed to its credit and move toward a more monetary policy",
 'a reuter survey of 17 money market economists produced a forecast of a 600 mln dlr m 1 decline for the week ended june 8 with estimates ranging from a gain of one billion dlrs to a decline of four billion',
 'instead m 1 fell eight bil

['allwaste lt o to acquire firm allwaste inc said it entered into an agreement in principle to buy co a privately held company that in exchange for about 1 3 mln shares or allwaste common',
 'allwaste said that earned about one mln dlrs before taxes last year and had about 1 5 mln dlrs in pre tax income for the nine months ended september 30',
 'fed data suggest no change in monetary policy new u s banking data suggest the federal reserve is monetary policy along a steady path and is not any imminent change of course economists said',
 "but they also said that if money supply growth remains weak as this week ' s unexpected eight billion dlr m 1 decline suggests it may this could influence the fed to its credit and move toward a more monetary policy",
 'a reuter survey of 17 money market economists produced a forecast of a 600 mln dlr m 1 decline for the week ended june 8 with estimates ranging from a gain of one billion dlrs to a decline of four billion',
 'instead m 1 fell eight billi

In [0]:
import tensorflow.contrib.keras as keras
import tensorflow as tf

from keras.engine import Layer, InputSpec
from keras import regularizers, initializers, constraints
from keras import backend as K

class AttentionWithContext(Layer):

    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    :param kwargs:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    refer https://github.com/fchollet/keras/issues/4962
    refer https://gist.github.com/rmdort/596e75e864295365798836d9e8636033
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.kernel = self.add_weight((input_shape[2], 1,),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        # word context vector uw
        self.u = self.add_weight((input_shape[1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        # in the paper refer equations (5) on page 3
        # (batch, time_steps, 40) x (40, 1)
        W_w_dot_h_it =  K.dot(x, self.kernel) # (batch, 40, 1)
        W_w_dot_h_it = K.squeeze(W_w_dot_h_it, -1) # (batch, 40)
        W_w_dot_h_it = W_w_dot_h_it + self.b # (batch, 40) + (40,)
        uit = K.tanh(W_w_dot_h_it) # (batch, 40)

        # in the paper refer equations (6) on page 3
        uit_dot_uw = uit * self.u # (batch, 40) * (40, 1) => (batch, 1)
        ait = K.exp(uit_dot_uw) # (batch, 1)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            mask = K.cast(mask, K.floatx()) #(batch, 40)
            ait = mask*ait #(batch, 40) * (batch, 40, )

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number epsilon to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = x * ait
        # sentence vector si is returned
        return K.sum(weighted_input, axis=1)

    def get_output_shape_for(self, input_shape):
        return input_shape[0], input_shape[-1]

    def compute_output_shape(self, input_shape):
        """Shape transformation logic so Keras can infer output shape
        """
        return (input_shape[0], input_shape[-1],)

In [0]:
from keras.preprocessing.text import Tokenizer
from nltk.corpus import reuters
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import sequence 
from itertools import chain

categories = [(cat, len(reuters.fileids(categories=cat))) for cat in reuters.categories()]
topn = [cat for cat, _ in sorted(categories, key=lambda x: -x[1])[:5]]

def iter_labels(selection='train'):
    for fid in reuters.fileids():
        if fid.startswith(selection):
            for sent in reuters.sents(fid):
                cat = reuters.categories(fid)[0]
                if cat in topn:
                  yield cat
labels_train = np.array(list(iter_labels('train')))
labels_test = np.array(list(iter_labels('test')))

def iter_sents(selection='train'):
    for fid in reuters.fileids():
        if fid.startswith(selection):
            for sent in reuters.sents(fid):
                cat = reuters.categories(fid)[0]
                if cat in topn:
                  yield " ".join(sent)
data_train = np.array(list(iter_sents('train')))
data_test = np.array(list(iter_sents('test')))

index_train = np.arange(data_train.shape[0])
index_test = np.arange(data_test.shape[0])

np.random.seed(1)
np.random.shuffle(index_train)
np.random.shuffle(index_test)

n_train = 200
n_test = 100

labels_train = labels_train[index_train][:n_train]
data_train = data_train[index_train][:n_train]

labels_test = labels_test[index_test][:n_test]
data_test = data_test[index_test][:n_test]

le = LabelEncoder().fit(topn)
y_train = le.transform(labels_train)
y_test = le.transform(labels_test)

y_train_hat = to_categorical(y_train, le.classes_.shape[0])
y_test_hat = to_categorical(y_test, le.classes_.shape[0])

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(data_train)

X_train = tokenizer.texts_to_sequences(data_train)
X_test = tokenizer.texts_to_sequences(data_test)

X_train = sequence.pad_sequences(X_train, maxlen=MAX_SEQUENCE_LEN)
X_test = sequence.pad_sequences(X_test, maxlen=MAX_SEQUENCE_LEN)

In [82]:
from keras.models import clone_model
copy = clone_model(model)
#encoder = AttentionWithContext()(copy.get_layer('tempconv1').output)
encoder = Lambda(lambda x: x[:, -1, :])(copy.get_layer('tempconv1').output)
classification_layer = Dense(le.classes_.shape[0], activation='softmax')(encoder)
clfr = Model(copy.input, classification_layer)
optimizer = optimizers.Adam(lr=0.002, clipnorm=0.4)
clfr.compile(optimizer=optimizer, metrics=['accuracy'], loss='categorical_crossentropy')
#print(clfr.summary())
history = clfr.fit(X_train, y_train_hat,
                   validation_data=(X_test, y_test_hat), shuffle=True, epochs=10, batch_size=32)

from sklearn.metrics import classification_report
y_test_pred = np.argmax(clfr.predict(X_test, verbose=1), axis=1)
y_train_pred = np.argmax(clfr.predict(X_train, verbose=1), axis=1)
print("TRAINING REPORT")
print(classification_report(y_train, y_train_pred, target_names=le.classes_))
print("VALIDATION REPORT")
print(classification_report(y_test, y_test_pred, target_names=le.classes_))

Train on 200 samples, validate on 100 samples
Epoch 1/10
200/200 [==============================] - 14s 69ms/step - loss: 1.5582 - acc: 0.3150 - val_loss: 1.4462 - val_acc: 0.3300
Epoch 2/10
200/200 [==============================] - 2s 8ms/step - loss: 1.3115 - acc: 0.4700 - val_loss: 1.5106 - val_acc: 0.3800
Epoch 3/10
200/200 [==============================] - 2s 8ms/step - loss: 1.1392 - acc: 0.5100 - val_loss: 1.3796 - val_acc: 0.3700
Epoch 4/10
200/200 [==============================] - 2s 8ms/step - loss: 0.9598 - acc: 0.6650 - val_loss: 1.4328 - val_acc: 0.4400
Epoch 5/10
200/200 [==============================] - 2s 8ms/step - loss: 0.7066 - acc: 0.7350 - val_loss: 1.4366 - val_acc: 0.4500
Epoch 6/10
200/200 [==============================] - 2s 8ms/step - loss: 0.4116 - acc: 0.9300 - val_loss: 1.8957 - val_acc: 0.4600
Epoch 7/10
200/200 [==============================] - 2s 8ms/step - loss: 0.3776 - acc: 0.8950 - val_loss: 2.3900 - val_acc: 0.4600
Epoch 8/10
200/200 [========

In [84]:
input_layer = Input(shape=(MAX_SEQUENCE_LEN,))
encoder = None
if not USE_GLOVE:
  encoder = Embedding(MAX_NUM_WORDS, embedding_size)(input_layer)
else:
  encoder = Embedding(num_words, embdedding_dim, 
                      input_length=MAX_SEQUENCE_LEN, 
                      embeddings_initializer=Constant(embedding_matrix),
                      trainable=True)(input_layer)
encoder = encoders[encoder_type](encoder)
#encoder = AttentionWithContext()(encoder)
encoder = Lambda(lambda x: x[:, -1, :])(encoder)
classification_layer = Dense(le.classes_.shape[0], activation='softmax')(encoder)
clfr2 = Model(input_layer, classification_layer)
optimizer = optimizers.Adam(lr=0.001, clipnorm=0.1)
clfr2.compile(optimizer=optimizer, metrics=['accuracy'], loss='categorical_crossentropy')
#print(clfr.summary())
history = clfr2.fit(X_train, y_train_hat,
                   validation_data=(X_test, y_test_hat), shuffle=True, epochs=10, batch_size=32)

from sklearn.metrics import classification_report
y_test_pred = np.argmax(clfr2.predict(X_test, verbose=1), axis=1)
y_train_pred = np.argmax(clfr2.predict(X_train, verbose=1), axis=1)
print("TRAINING REPORT")
print(classification_report(y_train, y_train_pred, target_names=le.classes_))
print("VALIDATION REPORT")
print(classification_report(y_test, y_test_pred, target_names=le.classes_))

Train on 200 samples, validate on 100 samples
Epoch 1/10
200/200 [==============================] - 14s 70ms/step - loss: 1.8311 - acc: 0.2500 - val_loss: 1.7896 - val_acc: 0.2300
Epoch 2/10
200/200 [==============================] - 2s 8ms/step - loss: 1.4612 - acc: 0.4150 - val_loss: 1.7072 - val_acc: 0.3100
Epoch 3/10
200/200 [==============================] - 2s 8ms/step - loss: 1.2399 - acc: 0.5100 - val_loss: 1.6310 - val_acc: 0.3200
Epoch 4/10
200/200 [==============================] - 2s 8ms/step - loss: 1.0735 - acc: 0.5700 - val_loss: 1.6036 - val_acc: 0.3400
Epoch 5/10
200/200 [==============================] - 2s 8ms/step - loss: 0.9271 - acc: 0.6600 - val_loss: 1.5907 - val_acc: 0.3600
Epoch 6/10
200/200 [==============================] - 2s 8ms/step - loss: 0.7991 - acc: 0.7350 - val_loss: 1.5952 - val_acc: 0.3600
Epoch 7/10
200/200 [==============================] - 2s 8ms/step - loss: 0.6826 - acc: 0.8050 - val_loss: 1.6068 - val_acc: 0.3500
Epoch 8/10
200/200 [========